In [1]:
#Import modules:
import sys
import os, errno
import user_interface as ui
import matplotlib.pyplot as plt
import iris
import iris.plot as iplt



from __future__ import (absolute_import, division, print_function)
from six.moves import (filter, input, map, range, zip)  # noqa



In [2]:
cubes = iris.load('../../coads_climatology.nc.nc')

In [3]:
print(cubes)

0: ZONAL WIND / (meter per second)     (TIME: 12; latitude: 90; longitude: 180)
1: AIR TEMPERATURE / (degree_celsius)  (TIME: 12; latitude: 90; longitude: 180)
2: SEA SURFACE TEMPERATURE / (degree_celsius) (TIME: 12; latitude: 90; longitude: 180)
3: MERIDIONAL WIND / (meter per second) (TIME: 12; latitude: 90; longitude: 180)


In [4]:
for c in cubes:
    if c.name() == 'AIR TEMPERATURE':
        c0 = c
print(c0)


AIR TEMPERATURE / (degree_celsius)  (TIME: 12; latitude: 90; longitude: 180)
     Dimension coordinates:
          TIME                           x             -              -
          latitude                       -             x              -
          longitude                      -             -              x
     Attributes:
          DODS_EXTRA.Unlimited_Dimension: TIME
          NC_GLOBAL.history: FERRET V4.30 (debug/no GUI) 15-Aug-96
          history: From coads_climatology


In [5]:
for coord in c0.coords():
    print(coord.name())

TIME
latitude
longitude


In [6]:
print(c0.coord('latitude'))
print(c0.coord('longitude'))

DimCoord(array([-89., -87., -85., -83., -81., -79., -77., -75., -73., -71., -69.,
       -67., -65., -63., -61., -59., -57., -55., -53., -51., -49., -47.,
       -45., -43., -41., -39., -37., -35., -33., -31., -29., -27., -25.,
       -23., -21., -19., -17., -15., -13., -11.,  -9.,  -7.,  -5.,  -3.,
        -1.,   1.,   3.,   5.,   7.,   9.,  11.,  13.,  15.,  17.,  19.,
        21.,  23.,  25.,  27.,  29.,  31.,  33.,  35.,  37.,  39.,  41.,
        43.,  45.,  47.,  49.,  51.,  53.,  55.,  57.,  59.,  61.,  63.,
        65.,  67.,  69.,  71.,  73.,  75.,  77.,  79.,  81.,  83.,  85.,
        87.,  89.]), standard_name='latitude', units=Unit('degrees'), var_name='COADSY', attributes={'point_spacing': 'even'})
DimCoord(array([  21.,   23.,   25.,   27.,   29.,   31.,   33.,   35.,   37.,
         39.,   41.,   43.,   45.,   47.,   49.,   51.,   53.,   55.,
         57.,   59.,   61.,   63.,   65.,   67.,   69.,   71.,   73.,
         75.,   77.,   79.,   81.,   83.,   85.,   87.,   89.

In [7]:
#This will give error:
#ValueError: 
#zero not allowed as a reference year, does not exist in Julian or Gregorian calendars

print(c0.coord('TIME'))

#Why? ...See the error and next box for the solution.

ValueError: zero not allowed as a reference year, does not exist in Julian or Gregorian calendars

In [8]:
#Reason for the error above: The reference year is set to zero...
#See...
print(c0.coord('TIME').units)

hour since 0000-01-01 00:00:00


In [9]:
#We change the reference year by doing the following:
c0.coord('TIME').units='hour since epoch'
c0t = c0.coord('TIME')
#"epoch" = 1st of january of 1970.
print(c0t.units)


hour since 1970-01-01 00:00:00


In [10]:
#Now we can safely print the coordinate:
print(c0.coord('TIME'))

DimCoord([1970-01-16 06:00:00, 1970-02-15 16:29:06.000004,
       1970-03-18 02:58:12, 1970-04-17 13:27:18, 1970-05-17 23:56:24,
       1970-06-17 10:25:30.000009, 1970-07-17 20:54:36,
       1970-08-17 07:23:42, 1970-09-16 17:52:48.000003,
       1970-10-17 04:21:54, 1970-11-16 14:51:00, 1970-12-17 01:20:06], standard_name=None, calendar='gregorian', var_name='TIME', attributes={'modulo': ' ', 'time_origin': '1-JAN-0000 00:00:00'})


In [11]:
#As the 'time_origin' attribute is still wrong...
print(c0.coord('TIME').attributes['time_origin'])

#We change it by:
c0.coord('TIME').attributes['time_origin'] = '01-JAN-1970 00:00:00'

print(c0.coord('TIME').attributes['time_origin'])
#temp = c2.extract(iris.Constraint(TIME=c'1970-01-16 06:00:00',longitude=45))
#print(temp)

1-JAN-0000 00:00:00
01-JAN-1970 00:00:00


In [12]:
#Now we're almost set to plot the evolution of the temperature in time...
#We need to extract a cube and for this we create constraints:
print("The original cube:\n {}\n".format(c0))
c1 = c0.extract(iris.Constraint(longitude=45))
print("Extracting a sub-cube with only longitude = 45:\n {}\n".format(c1))
c2 = c1.extract(iris.Constraint(latitude=47))
print("Extracting again a sub-cube with only latitude = 47:\n {}\n".format(c2))

The original cube:
 AIR TEMPERATURE / (degree_celsius)  (TIME: 12; latitude: 90; longitude: 180)
     Dimension coordinates:
          TIME                           x             -              -
          latitude                       -             x              -
          longitude                      -             -              x
     Attributes:
          DODS_EXTRA.Unlimited_Dimension: TIME
          NC_GLOBAL.history: FERRET V4.30 (debug/no GUI) 15-Aug-96
          history: From coads_climatology

Extracting a sub-cube with only longitude = 45:
 AIR TEMPERATURE / (degree_celsius)  (TIME: 12; latitude: 90)
     Dimension coordinates:
          TIME                           x             -
          latitude                       -             x
     Scalar coordinates:
          longitude: 45.0 degrees
     Attributes:
          DODS_EXTRA.Unlimited_Dimension: TIME
          NC_GLOBAL.history: FERRET V4.30 (debug/no GUI) 15-Aug-96
          history: From coads_climatology



In [13]:
#Another way of slicing a cube:
c3 = c0[:,45,80]

print(c3)
print(c0.data)



AIR TEMPERATURE / (degree_celsius)  (TIME: 12)
     Dimension coordinates:
          TIME                           x
     Scalar coordinates:
          latitude: 1.0 degrees
          longitude: 181.0 degrees
     Attributes:
          DODS_EXTRA.Unlimited_Dimension: TIME
          NC_GLOBAL.history: FERRET V4.30 (debug/no GUI) 15-Aug-96
          history: From coads_climatology
[[[-- -- -- ..., -- -- --]
  [-- -- -- ..., -- -- --]
  [-- -- -- ..., -- -- --]
  ..., 
  [-- -- -- ..., -- -- --]
  [-- -- -- ..., -- -- --]
  [-- -- -- ..., -- -- --]]

 [[-- -- -- ..., -- -- --]
  [-- -- -- ..., -- -- --]
  [-- -- -- ..., -- -- --]
  ..., 
  [-- -- -- ..., -- -- --]
  [-- -- -- ..., -- -- --]
  [-- -- -- ..., -- -- --]]

 [[-- -- -- ..., -- -- --]
  [-- -- -- ..., -- -- --]
  [-- -- -- ..., -- -- --]
  ..., 
  [-- -- -- ..., -- -- --]
  [-- -- -- ..., -- -- --]
  [-- -- -- ..., -- -- --]]

 ..., 
 [[-- -- -- ..., -- -- --]
  [-- -- -- ..., -- -- --]
  [-- -- -- ..., -- -- --]
  ..., 
  [--

In [ ]:
#We can now plot the temporal evolution on that (45,47) cell...
iplt.plot(c0.coord('TIME'),c0.coord('TIME'))
plt.show()